In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 读取原始数据
try:
    df = pd.read_csv('data.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('data.csv', encoding='latin-1')

print(f"\nData shape: {df.shape}")
print(f"Total records: {len(df):,}")
print(f"\nFirst 5 rows:")
df.head()


Data shape: (541909, 8)
Total records: 541,909

First 5 rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
# 检查原始数据的质量问题

# 1. 检查缺失值
print("\n1. Missing Values Check:")
print("-" * 60)
missing_values = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage (%)': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0])

# 2. 检查数量 <= 0 的记录
print("\n2. Quantity Anomaly Check:")
print("-" * 60)
zero_or_negative_qty = df[df['Quantity'] <= 0]
print(f"Records with Quantity <= 0: {len(zero_or_negative_qty):,} ({len(zero_or_negative_qty)/len(df)*100:.2f}%)")
if len(zero_or_negative_qty) > 0:
    print(f"Sample of anomalous quantities:")
    print(zero_or_negative_qty[['InvoiceNo', 'StockCode', 'Description', 'Quantity']].head())

# 3. 检查价格 <= 0 的记录
print("\n3. Price Anomaly Check:")
print("-" * 60)
zero_or_negative_price = df[df['UnitPrice'] <= 0]
print(f"Records with Price <= 0: {len(zero_or_negative_price):,} ({len(zero_or_negative_price)/len(df)*100:.2f}%)")
if len(zero_or_negative_price) > 0:
    print(f"Sample of anomalous prices:")
    print(zero_or_negative_price[['InvoiceNo', 'StockCode', 'Description', 'UnitPrice']].head())

# 4. 检查特殊的StockCode
print("\n4. StockCode Format Check:")
print("-" * 60)
# 检查包含特殊字符的StockCode
special_codes = df[df['StockCode'].astype(str).str.contains('[^A-Za-z0-9]', regex=True)]
print(f"StockCode records with special characters: {len(special_codes):,}")

# 检查常见的非商品代码
non_product_keywords = ['POST', 'D', 'C2', 'M', 'BANK', 'PADS', 'DOT']
non_product_mask = df['StockCode'].astype(str).str.upper().isin(non_product_keywords)
print(f"Suspected non-product codes (postage, etc.): {non_product_mask.sum():,}")
if non_product_mask.sum() > 0:
    print("\nSample of non-product codes:")
    print(df[non_product_mask][['StockCode', 'Description']].drop_duplicates().head(10))

# 5. 检查重复记录
print("\n5. Duplicate Records Check:")
print("-" * 60)
duplicates = df.duplicated()
print(f"Completely duplicate records: {duplicates.sum():,}")

print("\n" + "=" * 60)


1. Missing Values Check:
------------------------------------------------------------
             Missing Count  Missing Percentage (%)
Description           1454                    0.27
CustomerID          135080                   24.93

2. Quantity Anomaly Check:
------------------------------------------------------------
Records with Quantity <= 0: 10,624 (1.96%)
Sample of anomalous quantities:
    InvoiceNo StockCode                       Description  Quantity
141   C536379         D                          Discount        -1
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24

3. Price Anomaly Check:
------------------------------------------------------------
Records with Price <= 0: 2,517 (0.46%)
Sample of anomalous prices:
     InvoiceNo StockCode Descriptio

In [4]:
# 完整的数据预处理
print("=" * 60)
print("Starting Data Preprocessing...")
print("=" * 60)

df_processed = df.copy()
print(f"\nOriginal data: {len(df_processed):,} records")

# 1. 删除取消的订单 (C开头)
df_processed = df_processed[~df_processed['InvoiceNo'].astype(str).str.startswith('C')]
print(f"1. After removing cancelled orders: {len(df_processed):,} records (removed {len(df) - len(df_processed):,})")

# 2. 删除数量 <= 0 的记录
before = len(df_processed)
df_processed = df_processed[df_processed['Quantity'] > 0]
print(f"2. After removing Quantity <= 0: {len(df_processed):,} records (removed {before - len(df_processed):,})")

# 3. 删除价格 <= 0 的记录
before = len(df_processed)
df_processed = df_processed[df_processed['UnitPrice'] > 0]
print(f"3. After removing Price <= 0: {len(df_processed):,} records (removed {before - len(df_processed):,})")

# 4. 删除非商品代码 (POST, D, C2, M, BANK, PADS, DOT等)
before = len(df_processed)
non_product_keywords = ['POST', 'D', 'C2', 'M', 'BANK', 'PADS', 'DOT', 'CRUK', 'TEST']
mask = ~df_processed['StockCode'].astype(str).str.upper().isin(non_product_keywords)
df_processed = df_processed[mask]
print(f"4. After removing non-product codes: {len(df_processed):,} records (removed {before - len(df_processed):,})")

# 5. 删除重复记录
before = len(df_processed)
df_processed = df_processed.drop_duplicates()
print(f"5. After removing duplicates: {len(df_processed):,} records (removed {before - len(df_processed):,})")

# 6. 删除StockCode为空或异常的记录
before = len(df_processed)
df_processed = df_processed[df_processed['StockCode'].notna()]
df_processed = df_processed[df_processed['StockCode'].astype(str).str.strip() != '']
print(f"6. After removing empty StockCode: {len(df_processed):,} records (removed {before - len(df_processed):,})")

print(f"\n{'=' * 60}")
print(f"Preprocessing Complete!")
print(f"{'=' * 60}")
print(f"Retention rate: {len(df_processed)/len(df)*100:.2f}%")
print(f"Final data: {len(df_processed):,} records")

Starting Data Preprocessing...

Original data: 541,909 records
1. After removing cancelled orders: 532,621 records (removed 9,288)
2. After removing Quantity <= 0: 531,285 records (removed 1,336)
3. After removing Price <= 0: 530,104 records (removed 1,181)
4. After removing non-product codes: 527,806 records (removed 2,298)
4. After removing non-product codes: 527,806 records (removed 2,298)
5. After removing duplicates: 522,585 records (removed 5,221)
6. After removing empty StockCode: 522,585 records (removed 0)

Preprocessing Complete!
Retention rate: 96.43%
Final data: 522,585 records
5. After removing duplicates: 522,585 records (removed 5,221)
6. After removing empty StockCode: 522,585 records (removed 0)

Preprocessing Complete!
Retention rate: 96.43%
Final data: 522,585 records


In [5]:
# 计算预处理后的关键指标
print("=" * 60)
print("Data Statistics After Preprocessing")
print("=" * 60)

unique_invoices = df_processed['InvoiceNo'].nunique()
unique_items = df_processed['StockCode'].nunique()
items_per_transaction = df_processed.groupby('InvoiceNo')['StockCode'].count()
avg_transaction_width = items_per_transaction.mean()
median_transaction_width = items_per_transaction.median()

print(f"\nNumber of Transactions: {unique_invoices:,}")
print(f"Number of Items: {unique_items:,}")
print(f"Average Transaction Width: {avg_transaction_width:.2f} items/transaction")
print(f"Median Transaction Width: {median_transaction_width:.0f} items/transaction")
print(f"Total Records: {len(df_processed):,}")

# 商品频率分布
item_frequency = df_processed['StockCode'].value_counts()
print(f"\nItem Frequency Distribution:")
print(f"   - Most common item appears: {item_frequency.max():,} times")
print(f"   - Average appearance: {item_frequency.mean():.2f} times")
print(f"   - Median appearance: {item_frequency.median():.0f} times")

print(f"\nTransaction Width Distribution:")
print(items_per_transaction.describe())

Data Statistics After Preprocessing

Number of Transactions: 19,789
Number of Items: 3,916
Average Transaction Width: 26.41 items/transaction
Median Transaction Width: 15 items/transaction
Total Records: 522,585

Item Frequency Distribution:
   - Most common item appears: 2,253 times
   - Average appearance: 133.45 times
   - Median appearance: 65 times

Transaction Width Distribution:
count    19789.000000
mean        26.407853
std         47.327653
min          1.000000
25%          6.000000
50%         15.000000
75%         29.000000
max       1113.000000
Name: StockCode, dtype: float64


In [6]:
# 创建输出目录
output_dir = "datasets"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")

# 构建交易-商品映射
transactions_dict = df_processed.groupby('InvoiceNo')['StockCode'].apply(list).to_dict()
print(f"Transaction mapping built: {len(transactions_dict):,} transactions")

Output directory: datasets
Transaction mapping built: 19,789 transactions
Transaction mapping built: 19,789 transactions


In [7]:
# 定义8个数据集的配置
datasets_config = [
    # 实验组1: 改变交易数量 (25%, 50%, 75%, 100%)
    {"name": "dataset1_trans_25%", "transaction_ratio": 0.25, "strategy": "vary_transactions"},
    {"name": "dataset2_trans_50%", "transaction_ratio": 0.50, "strategy": "vary_transactions"},
    {"name": "dataset3_trans_75%", "transaction_ratio": 0.75, "strategy": "vary_transactions"},
    {"name": "dataset4_trans_100%", "transaction_ratio": 1.00, "strategy": "vary_transactions"},
    
    # 实验组2: 改变商品种类 (25%, 50%, 75%, 100%)
    {"name": "dataset5_items_25%", "item_ratio": 0.25, "strategy": "vary_items"},
    {"name": "dataset6_items_50%", "item_ratio": 0.50, "strategy": "vary_items"},
    {"name": "dataset7_items_75%", "item_ratio": 0.75, "strategy": "vary_items"},
    {"name": "dataset8_items_100%", "item_ratio": 1.00, "strategy": "vary_items"},
]

print("Dataset Configuration:")
for i, config in enumerate(datasets_config, 1):
    print(f"  {i}. {config['name']}")

Dataset Configuration:
  1. dataset1_trans_25%
  2. dataset2_trans_50%
  3. dataset3_trans_75%
  4. dataset4_trans_100%
  5. dataset5_items_25%
  6. dataset6_items_50%
  7. dataset7_items_75%
  8. dataset8_items_100%


In [8]:
# 生成所有8个数据集
stats = []
np.random.seed(42)  # 设置随机种子，确保可重复性

print("=" * 60)
print("Starting Dataset Generation...")
print("=" * 60)

for config in datasets_config:
    dataset_name = config['name']
    strategy = config['strategy']
    
    print(f"\nGenerating {dataset_name}")
    
    if strategy == "vary_transactions":
        # 策略1: 改变交易数量，保持商品种类和平均交易宽度
        ratio = config['transaction_ratio']
        
        # 随机选择指定比例的交易
        selected_invoices = np.random.choice(
            list(transactions_dict.keys()),
            size=int(len(transactions_dict) * ratio),
            replace=False
        )
        
        # 筛选数据
        subset_df = df_processed[df_processed['InvoiceNo'].isin(selected_invoices)].copy()
        
    else:  # vary_items
        # 策略2: 改变商品种类，保持交易数量
        ratio = config['item_ratio']
        
        # 选择最频繁的商品（保证数据质量）
        top_items = item_frequency.head(int(unique_items * ratio)).index.tolist()
        
        # 筛选包含这些商品的数据
        subset_df = df_processed[df_processed['StockCode'].isin(top_items)].copy()
        
        # 如果交易数量差异过大，进行调整
        subset_invoices = subset_df['InvoiceNo'].unique()
        if len(subset_invoices) > unique_invoices * 1.1:
            selected_invoices = np.random.choice(
                subset_invoices,
                size=min(len(subset_invoices), unique_invoices),
                replace=False
            )
            subset_df = subset_df[subset_df['InvoiceNo'].isin(selected_invoices)].copy()
    
    # 计算子数据集统计信息
    subset_invoices = subset_df['InvoiceNo'].nunique()
    subset_items = subset_df['StockCode'].nunique()
    subset_items_per_trans = subset_df.groupby('InvoiceNo')['StockCode'].count()
    subset_avg_width = subset_items_per_trans.mean()
    
    print(f"   Transactions: {subset_invoices:,} ({subset_invoices/unique_invoices*100:.1f}%)")
    print(f"   Items: {subset_items:,} ({subset_items/unique_items*100:.1f}%)")
    print(f"   Average Width: {subset_avg_width:.2f}")
    print(f"   Total Records: {len(subset_df):,}")
    
    # 保存数据集
    output_path = os.path.join(output_dir, f"{dataset_name}.csv")
    subset_df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"   Saved")
    
    # 记录统计信息
    stats.append({
        'Dataset': dataset_name,
        'Strategy': strategy,
        'Transactions': subset_invoices,
        'Items': subset_items,
        'Avg_Width': round(subset_avg_width, 2),
        'Total_Records': len(subset_df),
        'Trans_Ratio': f"{subset_invoices/unique_invoices*100:.1f}%",
        'Items_Ratio': f"{subset_items/unique_items*100:.1f}%"
    })

print(f"\n{'=' * 60}")
print("All Datasets Generated Successfully!")
print("=" * 60)

Starting Dataset Generation...

Generating dataset1_trans_25%
   Transactions: 4,947 (25.0%)
   Items: 3,606 (92.1%)
   Average Width: 26.47
   Total Records: 130,955
   Saved

Generating dataset2_trans_50%
   Transactions: 9,894 (50.0%)
   Items: 3,780 (96.5%)
   Average Width: 25.54
   Total Records: 252,715
   Saved

Generating dataset2_trans_50%
   Transactions: 9,894 (50.0%)
   Items: 3,780 (96.5%)
   Average Width: 25.54
   Total Records: 252,715
   Saved

Generating dataset3_trans_75%
   Transactions: 14,841 (75.0%)
   Items: 3,876 (99.0%)
   Average Width: 26.78
   Total Records: 397,424
   Saved

Generating dataset3_trans_75%
   Transactions: 14,841 (75.0%)
   Items: 3,876 (99.0%)
   Average Width: 26.78
   Total Records: 397,424
   Saved

Generating dataset4_trans_100%
   Transactions: 19,789 (100.0%)
   Items: 3,916 (100.0%)
   Average Width: 26.41
   Total Records: 522,585
   Saved

Generating dataset4_trans_100%
   Transactions: 19,789 (100.0%)
   Items: 3,916 (100.0%)
   

In [9]:
# 创建统计汇总表
stats_df = pd.DataFrame(stats)

# 保存统计信息
stats_file = os.path.join(output_dir, 'datasets_statistics.csv')
stats_df.to_csv(stats_file, index=False, encoding='utf-8')
print(f"Statistics saved to: {stats_file}\n")

print("=" * 100)
print("Summary of All Datasets")
print("=" * 100)
stats_df

Statistics saved to: datasets\datasets_statistics.csv

Summary of All Datasets


,Dataset,Strategy,Transactions,Items,Avg_Width,Total_Records,Trans_Ratio,Items_Ratio
0,dataset1_trans_25%,vary_transactions,4947,3606,26.47,130955,25.0%,92.1%
1,dataset2_trans_50%,vary_transactions,9894,3780,25.54,252715,50.0%,96.5%
2,dataset3_trans_75%,vary_transactions,14841,3876,26.78,397424,75.0%,99.0%
3,dataset4_trans_100%,vary_transactions,19789,3916,26.41,522585,100.0%,100.0%
4,dataset5_items_25%,vary_items,19144,979,19.49,373205,96.7%,25.0%
5,dataset6_items_50%,vary_items,19538,1958,24.60,480580,98.7%,50.0%
6,dataset7_items_75%,vary_items,19726,2937,26.18,516518,99.7%,75.0%
7,dataset8_items_100%,vary_items,19789,3916,26.41,522585,100.0%,100.0%


In [10]:
# 分组展示
print("\n" + "=" * 80)
print("Experiment Group 1: Varying Transaction Count (Item count relatively stable)")
print("=" * 80)
group1 = stats_df[stats_df['Strategy'] == 'vary_transactions']
display(group1[['Dataset', 'Transactions', 'Items', 'Avg_Width', 'Total_Records']])

print("\n" + "=" * 80)
print("Experiment Group 2: Varying Item Count (Transaction count relatively stable)")
print("=" * 80)
group2 = stats_df[stats_df['Strategy'] == 'vary_items']
display(group2[['Dataset', 'Transactions', 'Items', 'Avg_Width', 'Total_Records']])


Experiment Group 1: Varying Transaction Count (Item count relatively stable)


,Dataset,Transactions,Items,Avg_Width,Total_Records
0,dataset1_trans_25%,4947,3606,26.47,130955
1,dataset2_trans_50%,9894,3780,25.54,252715
2,dataset3_trans_75%,14841,3876,26.78,397424
3,dataset4_trans_100%,19789,3916,26.41,522585



Experiment Group 2: Varying Item Count (Transaction count relatively stable)


,Dataset,Transactions,Items,Avg_Width,Total_Records
4,dataset5_items_25%,19144,979,19.49,373205
5,dataset6_items_50%,19538,1958,24.60,480580
6,dataset7_items_75%,19726,2937,26.18,516518
7,dataset8_items_100%,19789,3916,26.41,522585
